<a href="https://www.kaggle.com/code/mikedelong/visualize-with-sentence-transformer?scriptVersionId=157157525" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install --quiet sentence_transformers
print('installed sentence transformers package.')

installed sentence transformers package.


In [2]:
import pandas as pd
filename='/kaggle/input/mr-beast-most-viewed-yt-video-100k-comments/mrbeast.csv'
# for this EDA we are only interested in unique comments, so let's just load the comments column
df = pd.read_csv(filepath_or_buffer=filename, usecols=['Comment'])
# and let's drop any duplicates
df = df.drop_duplicates(ignore_index=True)
df['token count'] = df['Comment'].apply(func=lambda x: len(x.split()))
df.head()

,Comment,token count
0,"Like I said in the video, subscribe if you hav...",16
1,"Huge props to the set designers, everything wa...",10
2,Jimmy's videos have transformed into high-octa...,11
3,"this man is honestly too good for this world, ...",18
4,mr beast never fails to make good content,8


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Comment      388 non-null    object
 1   token count  388 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.2+ KB


388 unique rows out of 100k is interesting.

In [4]:
from plotly.express import histogram
histogram(data_frame=df, x='token count')

In [5]:
from arrow import now
from sentence_transformers import SentenceTransformer
model_start = now()
model = SentenceTransformer(model_name_or_path='all-MiniLM-L12-v2',)
print('model max sequence length: {}'.format(model.max_seq_length))
# the default is 128 and we only have a handful of inputs with more than 40 tokens
# so we are virtually guaranteed to capture everything 
embeddings = model.encode(df['Comment'].values.tolist())
print('model time: {}'.format(now() - model_start))

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

model max sequence length: 128


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

model time: 0:00:13.599443


In [6]:
from plotly.express import scatter
from umap import UMAP

umap_model = UMAP(n_components=2, random_state=2023, verbose=True, n_jobs=1)
df[['u0', 'u1']] = umap_model.fit_transform(X=embeddings)
scatter(data_frame=df, x='u0', y='u1', hover_name='Comment', height=900).show()

UMAP(n_jobs=1, random_state=2023, verbose=True)
Sun Dec 31 01:25:31 2023 Construct fuzzy simplicial set
Sun Dec 31 01:25:31 2023 Finding Nearest Neighbors
Sun Dec 31 01:25:34 2023 Finished Nearest Neighbor Search
Sun Dec 31 01:25:37 2023 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Sun Dec 31 01:25:38 2023 Finished embedding


We mostly get clusters based on language; we also get squid, Netflix, Jimmy, and set clusters in English.

In [7]:
def tag(arg:str) -> str:
    tokens = arg.lower().split()
    if 'squid' in tokens or '"squid' in tokens or '\'squid' in tokens:
        return 'squid'
    if 'netflix' in tokens or 'neflix' in tokens or 'netflix!' in tokens:
        return 'Netflix'
    if 'jimmy' in tokens or 'jimmy.' in tokens:
        return 'Jimmy'
    if 'set' in tokens or 'sets' in tokens:
        return 'set'
    return 'other'

df['tag'] = df['Comment'].apply(func=tag)
scatter(data_frame=df, x='u0', y='u1', hover_name='Comment', height=900, color='tag').show()